In [1]:
# Imports
import numpy as np
import pandas as pd
import csv 
import sys
import time
import os
import shutil
import PySimpleGUI as sg
import string

In [2]:
# Timestamp 

timestr = time.strftime("%Y%m%d-%H%M%S")


In [3]:
# Inputs
input_file_folder = "/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/ClariostarOutputs/Archive/"
input_file_name = "qubit_data_Marko_22112022.CSV"

# final desired vol (nL)
final_pool_vol = 10000

# well position of pool in output plate
pool_well_in_output_plate = "B1"

# minimum volume to be pipetted (nL)
min_pipetting_capacity = float(25) # for Echo

# minimum volume that can be pipetted 
vol_available_in_well = 10000 # for e.g, if there is 20 uL in the well, 5000 nL is what can be available as 15 uL is the minimum working range in the PP-0200 Echo plate)

# Beckman or Echo
BeckmanOrEcho = "Echo"


In [5]:
sg.theme('LightBrown9')
    
layout = [
        [sg.Text('Choose automated platform to use:\n\t- Echo: non genomic DNA\n\t- Beckman: genomic DNA\n\t- Beckman PacBio: Pooling for PacBio on Beckman')],
        [sg.Text('Desired platform name:')],
        [sg.DropDown(["Echo", "Beckman", "Beckman PacBio"], default_value="Echo")],
        [sg.Submit(), sg.Cancel()]
        ]

window = sg.Window('NGS Pooling', layout)
event, values = window.read()
window.maximize()
window.close()

BeckmanOrEcho = values[0]


In [66]:
# Create GUI

sg.theme('LightBrown9')


if BeckmanOrEcho == "Echo":
    layout = [
        [sg.Text('Please enter the desired parameters')],
        [sg.Text('Input file folder (ending with /)', size =(25, 1)), sg.InputText("/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/ClariostarOutputs/")],
        [sg.Text('Input file name', size =(25, 1)), sg.InputText("qubit_data_Marko_22112022.CSV")],
        [sg.Text('Output file folder (ending with /)', size =(25, 1)), sg.InputText("/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/")],
        [sg.Text('Final pool volume (nL)', size =(25, 1)), sg.InputText("1000")],
        [sg.Text('Pooling well position', size =(25, 1)), sg.InputText("B1")],
        [sg.Text('Minimum pipetting capacity (nL)', size =(25, 1)), sg.InputText("25")],
        [sg.Text('Sample volume available (nL)', size =(25, 1)), sg.InputText("5000")],
        [sg.Text('Exclude samples with concentration lower than (ng/uL)', size =(25, 1)), sg.InputText("0.35")],
        [sg.Submit(), sg.Cancel()]
    ]

elif BeckmanOrEcho == "Beckman" or "Beckman PacBio":
    layout = [
        [sg.Text('Please enter the desired parameters')],
        [sg.Text('Input file folder (ending with /)', size =(25, 1)), sg.InputText("/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/ClariostarOutputs/")],
        [sg.Text('Input file name', size =(25, 1)), sg.InputText("qubit_data_Marko_22112022.CSV")],
        [sg.Text('Output file folder (ending with /)', size =(25, 1)), sg.InputText("/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/")],
        [sg.Text('Final pool volume (uL)', size =(25, 1)), sg.InputText("10")],
        [sg.Text('Pooling well position', size =(25, 1)), sg.InputText("B1")],
        [sg.Text('Minimum pipetting capacity (uL)', size =(25, 1)), sg.InputText("0.5")],
        [sg.Text('Sample volume available (uL)', size =(25, 1)), sg.InputText("25")],
        [sg.Text('Exclude conc (ng/uL) lower than', size =(25, 1)), sg.InputText("0.35")],
        [sg.Submit(), sg.Cancel()]
    ]     


window = sg.Window('Robot File Generator For NGS Pooling', layout)
event, values = window.read()
window.maximize()
window.close()    

# Assigning values 
input_file_folder = values[0]
input_file_name = values[1]
output_files_folder = values[2]
final_pool_vol = float(values[3])
pool_well_in_output_plate = values[4]
min_pipetting_capacity = float(values[5])
vol_available_in_well = float(values[6])
min_threshold_values = float(values[7])

# Dataframes
input_file_df = pd.read_csv(input_file_folder + input_file_name)
input_file_df.rename(columns={input_file_df.columns[2]: "Concentration"},inplace=True)


FileNotFoundError: [Errno 2] File b'/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/ClariostarOutputs/qubit_data_Marko_22112022.CSV' does not exist: b'/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/ClariostarOutputs/qubit_data_Marko_22112022.CSV'

In [49]:
# Outputs

output_files_folder = output_files_folder + "NGSpooling_" + timestr
os.mkdir(output_files_folder)
output_files_folder = output_files_folder + "/"

output_file_name = "ProcessingDetails.csv"
output_file_df = pd.DataFrame(columns=['Sample Well ID','Sample Concentration','Sample Calculated Weight', 'Sample Calculated Weight Normalised', 'Sample Calculated Volume In Pool'])

if BeckmanOrEcho == "Echo":
    output_echo_file_name = "EchoFile.csv"
    output_echo_file_df = pd.DataFrame(columns=['Source Well','Destination Well','Transfer Volume'])
elif BeckmanOrEcho == "Beckman":
    output_beckman_file_name = "BeckmanFile.csv"
    output_beckman_file_df = pd.DataFrame(columns=['Source Well','Destination Well','Transfer Volume'])    
elif BeckmanOrEcho == "Beckman PacBio":
    output_beckmanPacBio_file_name = "BeckmanPacBioFile.csv"
    output_beckmanPacBio_file_df = pd.DataFrame(columns=['Source Well 384','Source Well 96','Source Plate 96','Destination Well','Transfer Volume']) 
    
output_report_name = "ExceptionsReport.csv"
output_report_df = pd.DataFrame(columns=['Sample Well ID','Sample Concentration','Comment'])

# Copy input file into the output folder
os.mkdir(output_files_folder + "InputFiles/")
shutil.copyfile(input_file_folder+input_file_name, output_files_folder + "InputFiles/" + "InputFile.csv")


'/Users/keltoumboukra/Desktop/Coding projects - Git/SamplePoolingForNGS/Files/NGSpooling_20230321-153219/InputFiles/InputFile.csv'

In [50]:
# Trim input data frame 

# Remove standards 
input_file_df = input_file_df[input_file_df["Content"].str.contains("Standard") == False]
input_file_df = input_file_df.reset_index(drop=True)

# Remove samples with negative concentrations and add to output report
for row in input_file_df.itertuples():
    
    Index = row [0]
    Well = row[1]
    Concentration = row[3]
    
    if Concentration <= min_threshold_values:
        output_report_df = output_report_df.append({'Sample Well ID': Well, 'Sample Concentration': Concentration, 'Comment': "Sample has negative value for concentration"}, ignore_index=True)
        input_file_df = input_file_df.drop(Index)

input_file_df = input_file_df.reset_index(drop=True)
output_report_df = output_report_df.reset_index(drop=True)

# Copy trimmed input file into the output folder
input_file_df.to_csv(output_files_folder + "InputFiles/" + "InputTrimmedFile.csv", index=False)

In [51]:
# Calculate Weights 

max_concentration = input_file_df['Concentration'].max()
sum_weights = float()

for row in input_file_df.itertuples():
    
    Index = row [0]
    Well = row[1]
    Concentration = row[3]

    sample_weight = max_concentration/Concentration
    sum_weights += sample_weight
    output_file_df = output_file_df.append({'Sample Well ID': Well, 'Sample Concentration': Concentration, 'Sample Calculated Weight': sample_weight}, ignore_index=True)  
    

In [52]:
# Calculate Normalised Weights 

sum_normalised_weights = float(0)
for row in input_file_df.itertuples():
    
    Index = row [0]
    Well = row[1]
    Concentration = row[3]
    
    sample_normalised_weight = output_file_df["Sample Calculated Weight"].iloc[Index]/sum_weights
    sum_normalised_weights += sample_normalised_weight # for QC, should be =1
    output_file_df.at[Index,'Sample Calculated Weight Normalised']=sample_normalised_weight

In [53]:
# Translate 384 coordinates to 96 coordinates for Beckman PacBio

def _translate(id):
  id = id -1
  if (id // 24) % 2 ==0:
    if id % 2==0:
        destinationrow = 1 + id % 24 //2
        destinationCol = chr(ord('@')+1+id // 48)
        return { "plate": 1, "well": destinationCol + str(destinationrow)}
    else:
        destinationrow = 1 + id % 24 //2
        destinationCol = chr(ord('@')+1+id // 48)
        return { "plate": 2, "well": destinationCol + str(destinationrow)}
  else:
    if id % 2==0:
        destinationrow =  1 +id % 24 //2
        destinationCol = chr(ord('@')+1+id // 48)
        return { "plate": 3, "well": destinationCol + str(destinationrow)}
    else:
        destinationrow =  1 +id % 24 //2
        destinationCol = chr(ord('@')+1+id // 48)
        return { "plate": 4, "well": destinationCol + str(destinationrow)}

# Generate dictionary to to convert 384 well number to 96-well plate number and well coordinate

x = list(map(_translate, [i+1 for i in range(0,384)]))

rows = [i for i in string.ascii_uppercase[:16]]
columns = [i for i in range(1, 24 + 1)]
wells = [letter + str(number) if number >= 10 else letter + "0" + str(number) for letter in rows for number in columns]


d1=dict(zip(wells,x))
d1

#print(d1["G10"])
#print(d1["G10"]["plate"])
#print(d1["G10"]["well"])



{'plate': 2, 'well': 'D5'}
2
D5


In [54]:
# Calculate Volume to pipette for each sample in the final pool

vol_in_pool = float()
sum_vol_in_pool = float(0)
fail_bool = bool(0)

for row in input_file_df.itertuples():
    
    Index = row [0]
    Well = row[1]
    Concentration = row[3]
    
    vol_in_pool = output_file_df["Sample Calculated Weight Normalised"].iloc[Index]*final_pool_vol 
    sum_vol_in_pool += vol_in_pool # for QC, should be = final_pool_vol  
    
    if vol_in_pool < min_pipetting_capacity:
        output_report_df = output_report_df.append({'Sample Well ID': Well, 'Sample Concentration': Concentration, 'Comment': "Volume to be pipetted for this well ({} nL) is smaller than pipetting capacity ({} nL). You MUST reprocess the file".format(vol_in_pool,min_pipetting_capacity)}, ignore_index=True)
        fail_bool = bool(1)
    elif vol_in_pool > vol_available_in_well:
        output_report_df = output_report_df.append({'Sample Well ID': Well, 'Sample Concentration': Concentration, 'Comment': "Volume to be pipetted for this well ({} nL) is larger than the volume available ({} nL). You MUST reprocess the file".format(vol_in_pool,vol_available_in_well)}, ignore_index=True)
        fail_bool = bool(1)
    else:
        output_file_df.at[Index,'Sample Calculated Volume In Pool']=vol_in_pool
        if BeckmanOrEcho == "Echo":
            output_echo_file_df = output_echo_file_df.append({'Source Well': Well, 'Destination Well': pool_well_in_output_plate, 'Transfer Volume': vol_in_pool}, ignore_index=True)
        elif BeckmanOrEcho == "Beckman":
            output_beckman_file_df = output_beckman_file_df.append({'Source Well': Well, 'Destination Well': pool_well_in_output_plate, 'Transfer Volume': vol_in_pool}, ignore_index=True)
        elif BeckmanOrEcho == "Beckman PacBio":
            output_beckmanPacBio_file_df = output_beckmanPacBio_file_df.append({'Source Well 384': Well,'Source Well 96': d1[Well]["well"],'Source Plate 96' : d1[Well]["plate"],'Destination Well' : pool_well_in_output_plate,'Transfer Volume': vol_in_pool}, ignore_index=True)


In [55]:
# Calculate final pool concentration for QC 

sum_concentrations = float(0)

for row in output_file_df.itertuples():
    
    Concentration = row[2]
    VolInPool = row[5]
    
    sum_concentrations += (Concentration*VolInPool)

final_pool_concentration = "%.3f"%(sum_concentrations / final_pool_vol)

In [56]:
# Export files and communicate outcome to user in GUI

os.mkdir(output_files_folder + "OutputFiles/")

if BeckmanOrEcho == "Echo":
    if fail_bool == bool(1):
        layout = [[sg.Text("Processing failed. The concentration of 1 sample or more is out of range and doesn't allow pooling with the settings entered. Check report file for details.")], [sg.Button("OK")]]
        window = sg.Window("Echo File Generator For NGS Pooling", layout)
    else:
        output_file_df.to_csv(output_files_folder + "OutputFiles/" + output_file_name, index=False)
        output_echo_file_df.to_csv(output_files_folder + "OutputFiles/" + output_echo_file_name, index=False)
        layout = [[sg.Text("Processing successful! Final pool concentration: {} ng/uL.".format(final_pool_concentration))], [sg.Button("OK")]]
        window = sg.Window("Echo File Generator For NGS Pooling", layout)

elif BeckmanOrEcho == "Beckman":
    if fail_bool == bool(1):
        layout = [[sg.Text("Processing failed. The concentration of 1 sample or more is out of range and doesn't allow pooling with the settings entered. Check report file for details.")], [sg.Button("OK")]]
        window = sg.Window("Beckman File Generator For NGS Pooling", layout)
    else:
        output_file_df.to_csv(output_files_folder + "OutputFiles/" + output_file_name, index=False)
        output_beckman_file_df = output_beckman_file_df.sort_values(by='Transfer Volume', ascending=False)
        output_beckman_file_df.to_csv(output_files_folder + "OutputFiles/" + output_beckman_file_name, index=False)
        layout = [[sg.Text("Processing successful! Final pool concentration: {} ng/uL.".format(final_pool_concentration))], [sg.Button("OK")]]
        window = sg.Window("Beckman File Generator For NGS Pooling", layout)
        
elif BeckmanOrEcho == "Beckman PacBio":
    if fail_bool == bool(1):
        layout = [[sg.Text("Processing failed. The concentration of 1 sample or more is out of range and doesn't allow pooling with the settings entered. Check report file for details.")], [sg.Button("OK")]]
        window = sg.Window("Beckman File Generator For NGS Pooling", layout)
    else:
        output_file_df.to_csv(output_files_folder + "OutputFiles/" + output_file_name, index=False)
        output_beckmanPacBio_file_df = output_beckmanPacBio_file_df.sort_values(by='Transfer Volume', ascending=False)
        output_beckmanPacBio_file_df.to_csv(output_files_folder + "OutputFiles/" + output_beckmanPacBio_file_name, index=False)
        layout = [[sg.Text("Processing successful! Final pool concentration: {} ng/uL.".format(final_pool_concentration))], [sg.Button("OK")]]
        window = sg.Window("Beckman PacBio File Generator For NGS Pooling", layout)

output_report_df.to_csv(output_files_folder + "OutputFiles/" + output_report_name, index=False)

while True:
    event, values = window.read()
    if event == "OK" or event == sg.WIN_CLOSED:
        break
window.close()
